In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime

# Function to fetch S&P 500 companies and industries
def fetch_sp500_companies():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    sp500_table = pd.read_html(url)
    sp500_df = sp500_table[0]
    return sp500_df[['Symbol', 'Security', 'GICS Sector', 'Headquarters Location']]

# Function to fetch additional company info (e.g., year of establishment)
def fetch_company_info(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info
    return {
        'Year of Establishment': info.get('yearFounded', None),
        'Country': info.get('country', None),
        'Region': info.get('region', None)
    }

# Function to fetch the financials (balance sheet, income statement, and cash flow) for a given stock ticker
def fetch_financials(ticker):
    try:
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet
        income_statement = stock.financials
        cash_flow = stock.cashflow
        market_cap = stock.info.get('marketCap', None)  # Fetch market cap for ranking
        # Fetch equity from balance sheet if available
        equity = balance_sheet.loc['Total Stockholder Equity'] if 'Total Stockholder Equity' in balance_sheet.index else None

        return balance_sheet, income_statement, cash_flow, market_cap, equity
    except Exception as e:
        print(f"Error fetching financial data for {ticker}: {e}")
        return None, None, None, None, None

# Fetch S&P 500 companies
sp500_companies = fetch_sp500_companies()

# Initialize DataFrames to store data
financial_data = pd.DataFrame(columns=['Date', 'Category', 'Stock Code', 'Metric', 'Amount', 'Industry', 'Rank in S&P 500', 'Rank in Industry'])
index_data = pd.DataFrame(columns=['Company Name', 'Stock Code', 'Year of Establishment', 'Rank in S&P 500', 'Industry'])
industry_summary = pd.DataFrame(columns=['Industry', 'Total Companies', 'Region', 'Country'])

# Fetch financials and additional info for each company
market_caps = {}

for index, row in sp500_companies.iterrows():
    symbol = row['Symbol']
    company_name = row['Security']
    industry = row['GICS Sector']
    print(f"Fetching data for {symbol} ({company_name})...")

    # Fetch company information
    company_info = fetch_company_info(symbol)

    # Fetch financial data
    balance_sheet, income_statement, cash_flow, market_cap, equity = fetch_financials(symbol)

    if market_cap:
        market_caps[symbol] = (market_cap, industry)

    # Add to index data (masterdata sheet)
    index_data = pd.concat([index_data, pd.DataFrame({
        'Company Name': [company_name],
        'Stock Code': [symbol],
        'Year of Establishment': [company_info['Year of Establishment']],
        'Rank in S&P 500': [None],  # To be filled later
        'Industry': [industry]
    })], ignore_index=True)

    # Process financial data and clean zero or missing values in the 'Amount' column
    if balance_sheet is not None and not balance_sheet.empty:
        # Iterate through all available columns (years) in the balance sheet
        for column in balance_sheet.columns:
            for item in balance_sheet.index:
                amount = balance_sheet.loc[item, column]
                if pd.notnull(amount) and amount != 0:  # Exclude missing or zero values
                    financial_data = pd.concat([financial_data, pd.DataFrame({
                        'Date': [column],  # Use the reporting date
                        'Category': ['Balance Sheet'],
                        'Stock Code': [symbol],
                        'Metric': [item],
                        'Amount': [amount],
                        'Industry': [industry],
                        'Rank in S&P 500': [None],  # To be filled later
                        'Rank in Industry': [None]  # To be filled later
                    })], ignore_index=True)

    if income_statement is not None and not income_statement.empty:
        for column in income_statement.columns:
            for item in income_statement.index:
                amount = income_statement.loc[item, column]
                if pd.notnull(amount) and amount != 0:  # Exclude missing or zero values
                    financial_data = pd.concat([financial_data, pd.DataFrame({
                        'Date': [column],  # Use the reporting date
                        'Category': ['Income Statement'],
                        'Stock Code': [symbol],
                        'Metric': [item],
                        'Amount': [amount],
                        'Industry': [industry],
                        'Rank in S&P 500': [None],  # To be filled later
                        'Rank in Industry': [None]  # To be filled later
                    })], ignore_index=True)

    if cash_flow is not None and not cash_flow.empty:
        for column in cash_flow.columns:
            for item in cash_flow.index:
                amount = cash_flow.loc[item, column]
                if pd.notnull(amount) and amount != 0:  # Exclude missing or zero values
                    financial_data = pd.concat([financial_data, pd.DataFrame({
                        'Date': [column],  # Use the reporting date
                        'Category': ['Cash Flow'],
                        'Stock Code': [symbol],
                        'Metric': [item],
                        'Amount': [amount],
                        'Industry': [industry],
                        'Rank in S&P 500': [None],  # To be filled later
                        'Rank in Industry': [None]  # To be filled later
                    })], ignore_index=True)

    if equity is not None and pd.notnull(equity) and equity != 0:
        # Add Equity data
        new_row = pd.DataFrame({
            'Date': [column],  # Use the reporting date
            'Category': ['Equity'],
            'Stock Code': [symbol],
            'Metric': ['Total Stockholder Equity'],
            'Amount': [equity],
            'Industry': [industry],
            'Rank in S&P 500': [None],  # To be filled later
            'Rank in Industry': [None]  # To be filled later
        })
        financial_data = pd.concat([financial_data, new_row], ignore_index=True)

# Calculate Rank in S&P 500 based on Market Cap
sorted_market_caps = sorted(market_caps.items(), key=lambda x: x[1][0], reverse=True)
rank_in_sp500 = {symbol: rank + 1 for rank, (symbol, _) in enumerate(sorted_market_caps)}

# Calculate Rank in Industry based on Market Cap
industry_groups = {}
for symbol, (market_cap, industry) in market_caps.items():
    if industry not in industry_groups:
        industry_groups[industry] = []
    industry_groups[industry].append((symbol, market_cap))

rank_in_industry = {}
for industry, companies in industry_groups.items():
    sorted_industry = sorted(companies, key=lambda x: x[1], reverse=True)
    for rank, (symbol, _) in enumerate(sorted_industry):
        rank_in_industry[symbol] = rank + 1

# Update ranks in the DataFrame
financial_data['Rank in S&P 500'] = financial_data['Stock Code'].map(rank_in_sp500)
financial_data['Rank in Industry'] = financial_data['Stock Code'].map(rank_in_industry)

# Save everything to Excel
with pd.ExcelWriter('sp500_financial_data_with_master_and_industry.xlsx', engine='xlsxwriter') as writer:
    financial_data.to_excel(writer, sheet_name='FinancialData', index=False)
    index_data.to_excel(writer, sheet_name='Index', index=False)
    industry_summary.to_excel(writer, sheet_name='IndustrySummary', index=False)

print("Data saved to 'sp500_financial_data_with_master_and_industry.xlsx'")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.2 MB/s eta 0:00:00
Fetching data for MMM (3M)...


<ipython-input-1-52fb16d9c59f>:82: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  financial_data = pd.concat([financial_data, pd.DataFrame({


Fetching data for AOS (A. O. Smith)...
Fetching data for ABT (Abbott Laboratories)...
Fetching data for ABBV (AbbVie)...
Fetching data for ACN (Accenture)...
Fetching data for ADBE (Adobe Inc.)...
Fetching data for AMD (Advanced Micro Devices)...
Fetching data for AES (AES Corporation)...
Fetching data for AFL (Aflac)...
Fetching data for A (Agilent Technologies)...
Fetching data for APD (Air Products)...
Fetching data for ABNB (Airbnb)...
Fetching data for AKAM (Akamai Technologies)...
Fetching data for ALB (Albemarle Corporation)...
Fetching data for ARE (Alexandria Real Estate Equities)...
Fetching data for ALGN (Align Technology)...
Fetching data for ALLE (Allegion)...
Fetching data for LNT (Alliant Energy)...
Fetching data for ALL (Allstate)...
Fetching data for GOOGL (Alphabet Inc. (Class A))...
Fetching data for GOOG (Alphabet Inc. (Class C))...
Fetching data for MO (Altria)...
Fetching data for AMZN (Amazon)...
Fetching data for AMCR (Amcor)...
Fetching data for AEE (Ameren)...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Data saved to 'sp500_financial_data_with_master_and_industry.xlsx'


In [ ]:
from google.colab import files

# Download the Excel file to your local machine
files.download('sp500_financial_data_with_master_and_industry.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>